In [1]:
import json
import numpy
import pandas
from pprint import pprint
pandas.options.display.max_colwidth = -1

In [2]:
# runs in about 5 minutes
with open('events_2012-2016.json','r') as f:
#with open('events_1927_1938.json','r') as f:
    allgames = json.load(f)
print('ok')
#pprint(allgames[2])   
allgames_bk = allgames

allgames_df = pandas.DataFrame(allgames)
allgames_df = allgames_df.set_index('id')

ok


In [3]:
#allgames_df.sample(10).index
mygames = ['NYN201204100', 'BAL201208060', 'SEA201208150', 'BAL201208300', 'BAL201209300', 'BAL201304210']
mygames += ['BAL201305170', 'BAL201405010', 'BAL201406130', 'BAL201406250', 'MIA201407030', 'PIT201605180']
mygames += ['SDN201606150', 'WAS201607200', 'BAL201608180']

#weird_game_ids = ['CLE192708190', 'BOS193008310', 'BOS193107250', 'CHA193304220', 'NYA193606030', 'MLN195505152']
#weird_game_ids += ['MIN196907170', 'NYN197208160', 'KCA201109150', 'TOR201208130', 'KCA201605310']

somecolumns = allgames_df.columns
allgames2_df = pandas.DataFrame(columns=somecolumns)
allgames2_df['id'] = ''
allgames2_df = allgames2_df.set_index('id')
allgames2_df
for thisid, thisrow in allgames_df.iterrows():
    if (thisid in mygames):
        allgames2_df = allgames2_df.append(thisrow)
allgames_df = allgames2_df
print('done')

done


In [ ]:
def parse_half_inning(old_df):
    
    new_df = old_df    
    new_df['play'] = old_df['event'].str.split('/').str.get(0)
    new_df['modifiers'] = old_df['event'].str.split('/',n=1).str.get(1)
    new_df['play'] = old_df['event'].str.split('/').str.get(0)
    new_df['modifiers'] = old_df['event'].str.split('/',n=1).str.get(1)
    new_df['modifiers'] = old_df['modifiers'].fillna('')
    new_df['baserunning'] = old_df['modifiers'].str.split('.',n=1).str.get(1)
    new_df['baserunning'] = old_df['baserunning'].fillna('')
    new_df['modifiers'] = old_df['modifiers'].str.split('.').str.get(0)
    
    return new_df

In [4]:
all_rosters_df = pandas.read_csv('fullroster.csv')
all_rosters_df.head(2)

def get_starters(onerow):
    
#    df = pandas.DataFrame()
    
    vsdf = pandas.DataFrame(onerow['visiting_starters'], columns=player_columns)
    vsdf['which_team'] = 0
    hsdf = pandas.DataFrame(onerow['home_starters'], columns=player_columns)
    hsdf['which_team'] = 1
    df = pandas.concat((vsdf, hsdf))
    df['batting_order'] = pandas.to_numeric(df['batting_order'])
    df['position'] = pandas.to_numeric(df['position'])

    return df

def split_event(df):
    df['play'] = df['event'].str.split('/').str.get(0)
    df['modifiers'] = df['event'].str.split('/',n=1).str.get(1)
    df['play'] = df['event'].str.split('/').str.get(0)
    df['modifiers'] = df['event'].str.split('/',n=1).str.get(1)
    df['modifiers'] = df['modifiers'].fillna('')
    df['baserunning'] = df['modifiers'].str.split('.',n=1).str.get(1)
    df['baserunning'] = df['baserunning'].fillna('')
    df['modifiers'] = df['modifiers'].str.split('.').str.get(0)
    
    return df

In [ ]:
column_list = ['event_in_game', 'event_type', 'inning', 'home_or_visitor', 'batterID']
column_list += ['pitch_count', 'pitch_sequence', 'event', 'comment', 'sub_batting_order', 'sub_position']

player_columns = ['playerID', 'name', 'batting_order', 'position']

for thisgameID,thisrow in allgames_df.iterrows():
    
    thisgame_df = pandas.DataFrame(thisrow['events'], columns=column_list)
    thisgame_df['gameID'] = thisgameID
    
    thisgame_df['inning'] = pandas.to_numeric(thisgame_df['inning'])
    thisgame_df['home_or_visitor'] = pandas.to_numeric(thisgame_df['home_or_visitor'])
    
    playing_df = get_starters(thisrow)

    thisgame_df = split_event(thisgame_df)
    
    thisgame_df = thisgame_df.set_index('event_in_game')
    
    for ing in thisgame_df['inning'].drop_duplicates():
        for side in range(0,2):
            halfinn = thisgame_df[(thisgame_df['inning'] == ing) & (thisgame_df['home_or_visitor'] == side)]
            for eig, thisevent in halfinn.iterrows():
                #print(eig, thisevent['play'])
                thisevent['pitcherID'] = playing_df['playerID'][(playing_df['which_team'] == 0) & (playing_df['position'] == 1)].tolist()[0]
                thisevent['catcherID'] = playing_df['playerID'][(playing_df['which_team'] == 0) & (playing_df['position'] == 2)].tolist()[0]
                thisevent['1bID'] = playing_df['playerID'][(playing_df['which_team'] == 0) & (playing_df['position'] == 3)].tolist()[0]
                thisevent['2bID'] = playing_df['playerID'][(playing_df['which_team'] == 0) & (playing_df['position'] == 4)].tolist()[0]
                thisevent['3bID'] = playing_df['playerID'][(playing_df['which_team'] == 0) & (playing_df['position'] == 5)].tolist()[0]
                thisevent['ssID'] = playing_df['playerID'][(playing_df['which_team'] == 0) & (playing_df['position'] == 6)].tolist()[0]
                thisevent['lfID'] = playing_df['playerID'][(playing_df['which_team'] == 0) & (playing_df['position'] == 7)].tolist()[0]
                thisevent['cfID'] = playing_df['playerID'][(playing_df['which_team'] == 0) & (playing_df['position'] == 8)].tolist()[0]
                thisevent['rfID'] = playing_df['playerID'][(playing_df['which_team'] == 0) & (playing_df['position'] == 9)].tolist()[0]
                
                print(thisevent[['event_type', 'batterID', 'pitcherID', 'catcherID', '1bID', '2bID', '3bID', 'ssID', 'lfID', 'cfID', 'rfID', 'play', 'baserunning']])
                print('----------')